In [1]:
import pandas as pd
import xgboost as xgb
import numpy as np

#
#import sklearn
#import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score

In [2]:
sample = pd.read_csv("sample_submission.csv",encoding = "ISO-8859-1")
sample.head(2)

,id,target
0,0,0
1,2,0


In [3]:
test_limpio = pd.read_csv("test_limpio_con_BOW_de_5000_y_Stemming.csv",encoding = "ISO-8859-1")
test_limpio.head()

,aa,ab,aba,abandon,abbott,abbswinston,abc,abcnew,abe,abia,...,money_related,tiene_fecha,has_year,tiene_hora_2,tiene_hora,repeticiones,longitud_tweet,letras_seguidas,sentimiento,objetividad
0,0,0,0,0,0,0,0,0,0,0,...,False,False,False,False,False,1,40,2,-1.00,1.00
1,0,0,0,0,0,0,0,0,0,0,...,False,False,False,False,False,1,70,2,0.25,0.55
2,0,0,0,0,0,0,0,0,0,0,...,False,False,False,False,False,1,100,2,0.00,0.00
3,0,0,0,0,0,0,0,0,0,0,...,False,False,False,False,False,1,40,1,0.00,0.00
4,0,0,0,0,0,0,0,0,0,0,...,False,False,False,False,False,1,50,2,0.00,0.00


In [4]:
train_limpio = pd.read_csv("train_limpio_con_BOW_de_5000_y_Stemming.csv",encoding = "ISO-8859-1")
train_limpio.head(2)

,aa,ab,aba,abandon,abbott,abbswinston,abc,abcnew,abe,abia,...,money_related,tiene_fecha,has_year,tiene_hora_2,tiene_hora,repeticiones,longitud_tweet,letras_seguidas,sentimiento,objetividad
0,0,0,0,0,0,0,0,0,0,0,...,False,False,False,False,False,1,70,2,0.0,0.0
1,0,0,0,0,0,0,0,0,0,0,...,False,False,False,False,False,1,40,1,0.1,0.4


In [5]:
df_y=train_limpio.target
df_y

0       1
1       1
2       1
3       1
4       1
5       1
6       1
7       1
8       1
9       1
10      1
11      1
12      1
13      1
14      1
15      0
16      0
17      0
18      0
19      0
20      0
21      0
22      0
23      0
24      0
25      0
26      0
27      0
28      0
29      0
       ..
6967    0
6968    0
6969    0
6970    0
6971    0
6972    0
6973    0
6974    0
6975    0
6976    1
6977    1
6978    0
6979    1
6980    1
6981    1
6982    1
6983    1
6984    0
6985    1
6986    1
6987    1
6988    1
6989    1
6990    1
6991    1
6992    1
6993    1
6994    1
6995    1
6996    1
Name: target, Length: 6997, dtype: int64

In [6]:
df_x=train_limpio.drop([ 'target'], axis=1)
df_x.head()

,aa,ab,aba,abandon,abbott,abbswinston,abc,abcnew,abe,abia,...,money_related,tiene_fecha,has_year,tiene_hora_2,tiene_hora,repeticiones,longitud_tweet,letras_seguidas,sentimiento,objetividad
0,0,0,0,0,0,0,0,0,0,0,...,False,False,False,False,False,1,70,2,0.00000,0.0000
1,0,0,0,0,0,0,0,0,0,0,...,False,False,False,False,False,1,40,1,0.10000,0.4000
2,0,0,0,0,0,0,0,0,0,0,...,False,False,False,False,False,1,130,2,-0.01875,0.3875
3,0,0,0,0,0,0,0,0,0,0,...,False,False,False,False,False,1,60,1,0.00000,0.0000
4,0,0,0,0,0,0,0,0,0,0,...,False,False,False,False,False,1,90,2,0.00000,0.0000


## Prueba basica

In [7]:
x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.2)

In [8]:
#rf= RandomForestClassifier(n_estimators=50)
#xg_reg = xgb.XGBClassifier(objective ='reg:squarederror', 
#                colsample_bytree = 0.3, learning_rate = 0.1,
#                max_depth = 5, alpha = 10, n_estimators = 10)
xgc = xgb.XGBClassifier()
xgc.fit(x_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [9]:
pred = xgc.predict(x_test)

In [10]:
f1_score(y_test.values, pred, average='macro')

0.7201012126846968

## Prueba con cross-validation

In [12]:
kf = KFold(n_splits = 5)
kf

KFold(n_splits=5, random_state=None, shuffle=False)

In [13]:
for train_index,test_index in kf.split(df_x):
    x_train,x_test = df_x.loc[train_index], df_x.loc[test_index]
    y_train,y_test = df_y[train_index],df_y[test_index]
    
    xgc = xgb.XGBClassifier()
    xgc.fit(x_train, y_train)
    
    #error de entrenamiento:
    pred = xgc.predict(x_train)
    print('Error de entrenamiento: ',f1_score(y_train.values, pred, average='macro'))
    
    
        
    #error de test:
    pred = xgc.predict(x_test)
    print('Error de test: ',f1_score(y_test.values, pred, average='macro'))
    

por entrenar
('Error de entrenamiento: ', 0.7564259445307925)
('Error de test: ', 0.6936710717629446)
por entrenar
('Error de entrenamiento: ', 0.755093620644625)
('Error de test: ', 0.6378418329637843)
por entrenar
('Error de entrenamiento: ', 0.7472538885115013)
('Error de test: ', 0.6608667953407388)
por entrenar
('Error de entrenamiento: ', 0.7541788949764179)
('Error de test: ', 0.6503295696428182)
por entrenar
('Error de entrenamiento: ', 0.7391639828234031)
('Error de test: ', 0.6630925074167504)


# tunning con griddsearch, masomenos andaba cuando tenia solo 1000 columnas, ahora imposible

In [27]:
params_grid={
    'n_estimators': [50,100,150,200],
    'learning_rate' : [0.3,0.45,0.6,0.75],
    'max_depth' : [3,6,9,12]
    
}

In [28]:
params_fixed={
    'objective':'binary:logistic',
    'silent' : 1
}

In [29]:
bst_grid= GridSearchCV(
    estimator=xgb.XGBClassifier(**params_fixed),
    param_grid=params_grid,
    cv=5,
    scoring='accuracy'
)

In [ ]:
bst_grid.fit(x_train, y_train)

In [ ]:
best_parameters = bst_grid.best_params_
print(best_parameters)

Dio que {'n_estimators': 100, 'learning_rate': 0.45, 'max_depth': 2}

In [1]:
# Este fue el resultado con los mejores hyperparametros de antes, ahora supongo que van a ser otros

In [15]:
for train_index,test_index in kf.split(df_x):
    x_train,x_test = df_x.loc[train_index], df_x.loc[test_index]
    y_train,y_test = df_y[train_index], df_y[test_index]
    
    xgc = xgb.XGBClassifier(n_estimatores=100 , learning_rate=0.45 , max_depth=6)
    xgc.fit(x_train, y_train)
    
    #error de entrenamiento:
    pred = xgc.predict(x_train)
    print('Error de entrenamiento: ',f1_score(y_train.values, pred, average='macro'))
    
    
        
    #error de test:
    pred = xgc.predict(x_test)
    print('Error de test: ',f1_score(y_test.values, pred, average='macro'))
    

('Error de entrenamiento: ', 0.88079232271373)
('Error de test: ', 0.7251644031216455)
('Error de entrenamiento: ', 0.8926956965131625)
('Error de test: ', 0.6918154765334887)
('Error de entrenamiento: ', 0.88341128847895)
('Error de test: ', 0.7103052835878971)
('Error de entrenamiento: ', 0.8889053333055272)
('Error de test: ', 0.6682714187736285)
('Error de entrenamiento: ', 0.8839820496606485)
('Error de test: ', 0.737337032554866)


In [119]:
#La realizo devuelta con mayor precision alrededor de los puntos seleccionados antes

In [118]:
params_grid={
    'n_estimators': [70,80,90,100,110,120,130],
    'learning_rate' : [0.35,0.4,0.45,0.5,0.55],
    'max_depth' : [1,2,3]
    
}

In [120]:
bst_grid= GridSearchCV(
    estimator=xgb.XGBClassifier(**params_fixed),
    param_grid=params_grid,
    cv=5,
    scoring='accuracy'
)

In [121]:
bst_grid.fit(df_x, df_y)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None, silent=1,
       subsample=1),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_estimators': [70, 80, 90, 100, 110, 120, 130], 'learning_rate': [0.35, 0.4, 0.45, 0.5, 0.55], 'max_depth': [1, 2, 3]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [122]:
best_parameters = bst_grid.best_params_
print(best_parameters)

{'n_estimators': 100, 'learning_rate': 0.45, 'max_depth': 2}


In [124]:
# Me da lo mismo, lo dejo aca

In [25]:
for train_index,test_index in kf.split(train_sin_texto):
    x_train,x_test = df_x.loc[train_index], df_x.loc[test_index]
    y_train,y_test = df_y[train_index], df_y[test_index]
    
    xgc = xgb.XGBClassifier(n_estimators=170,learning_rate=0.6, max_depth=300)
    xgc.fit(x_train, y_train)
    
    #error de entrenamiento:
    pred = xgc.predict(x_train)
    print('Error de entrenamiento: ',f1_score(y_train.values, pred, average='macro'))
    
    
        
    #error de test:
    pred = xgc.predict(x_test)
    print('Error de test: ',f1_score(y_test.values, pred, average='macro'))

('Error de entrenamiento: ', 0.990690743566668)
('Error de test: ', 0.7126520059491375)
('Error de entrenamiento: ', 0.9893737281175179)
('Error de test: ', 0.6882951846416847)
('Error de entrenamiento: ', 0.991220004243921)
('Error de test: ', 0.6642129800024537)
('Error de entrenamiento: ', 0.9911884059613871)
('Error de test: ', 0.6860695341845173)


KeyboardInterrupt: 

In [54]:
# Entonces, mi modelo optimizado es:
train_x=train_limpio.drop(['text', 'target'], axis=1)

train_y=train_limpio.target



xgc = xgb.XGBClassifier(n_estimators=170,learning_rate=0.6)
xgc.fit(train_x, train_y)
    
#error de entrenamiento, es el unico que puedo calcular:
pred = xgc.predict(train_x)
print('Error de entrenamiento: ',f1_score(train_y, pred, average='macro'))

('Error de entrenamiento: ', 0.8218694822450909)


In [69]:
pred = xgc.predict(test_limpio_sin_texto.drop(['id'],axis=1))

In [70]:
sample.target=pred

In [71]:
sample.to_csv('prediccion_xgboost.csv', index=False)